# MyAnimeList Recommendations Database analysis

## Step 1 : Loading the Data

Let's import all the modules we need for the anaysis and start loading the files we need into `pandas` dataframes from their respective `.csv` files.

First up is the **Anime metadata**, located in `../data/raw/anime.csv`:

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.style as style
import csv
import pandas_profiling as pprof

anime_df = pd.read_csv('../data/raw/anime.csv')
anime_df.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


Looking good!

Now let's do the same for the **User Ratings Data**, located in `../data/raw/rating.csv`:

In [14]:
ratings_df = pd.read_csv('../data/raw/rating.csv')
ratings_df.head()

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


Perfect! No issues so far, so the next step is...

## Step 2: Let's Dive into the dataset! (Initial Observations, Data cleaning, and general prep work)
### Initial observation through profile reports
The two `.html` files in the `analysis` folder are profile reports generated through the `pandas_profiling` module, which allows us to see a detailed description of the data and some preliminary statistics and metadata about the data as well. This will help us identify what types of values there are, and what values we need to clean up!

The Profiling Report file for each dataframe was created using the following code:

`pr = pprof.ProfileReport(ratings_df)
pr.to_file(output_file='ratings_profiling_report.html')
pr = pprof.ProfileReport(anime_df)
pr.to_file(output_file='anime_profiling_report.html')`

**NOTE:** *This is not included in the notebook since the files are quite large (around 100MB), and running this profiling locally each time would take too much time.*

### Data Cleanup
